<a href="https://colab.research.google.com/github/danielbauer1979/CAS_PredMod/blob/main/pa_pynb_sess4_LASSO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Lasso, LassoCV
from sklearn.metrics import mean_squared_error

In [ ]:
!git clone https://github.com/danielbauer1979/CAS_PredMod.git

In [5]:
dat_1 = pd.read_csv('CAS_PredMod/pa_data_Allstate_train1.csv')
dat_2 = pd.read_csv('CAS_PredMod/pa_data_Allstate_train2.csv')
dat_3 = pd.read_csv('CAS_PredMod/pa_data_Allstate_train3.csv')
df = pd.concat([dat_1,dat_2,dat_3])

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
df.describe()

In [9]:
df = df[df['loss']>100]

In [10]:
#Convert categoricals into dummies
del df['id']
objects = []
for c in df.columns:
    if str(df[c].dtype) == 'object':
        objects.append(c)
X_ = df.drop(objects, axis = 1).astype('float64')
X_ = X_.drop(['loss'], axis = 1)
dummies = pd.get_dummies(df[objects], drop_first=True)
X = pd.concat([X_, dummies], axis = 1)
y = df.loss

In [ ]:
X

In [12]:
# Split data into training and test sets
X_train, X_test , y_train, y_test = train_test_split(X, y, test_size=0.8, random_state=1)
X_val, X_test, y_val, y_test = train_test_split(X,y,test_size = 0.6, random_state=2)
y_train = np.log(y_train)
y_val = np.log(y_val)
y_test = np.log(y_test)

In [ ]:
model_ols = LinearRegression(fit_intercept=True)
model_ols.fit(X_train, y_train)
print(model_ols.intercept_)
print(model_ols.coef_)

In [ ]:
TestRMSE_ols = np.sqrt(mean_squared_error(y_test,model_ols.predict(X_test)))
print(TestRMSE_ols)

In [ ]:
tmp = model_ols.predict(X_test)-y_test
print(np.median(tmp))
print(np.quantile(tmp,0.9))
print(np.quantile(tmp,0.99))
print(np.quantile(tmp,0.999))
print(np.amax(tmp))

In [ ]:
alphas = np.array([0.000001, 0.000003, 0.000007, 0.00001, 0.0001, 0.001])
model_lasso = Lasso(max_iter = 10000, normalize = True)
coefs = []
MSE = []
for a in alphas:
    model_lasso.set_params(alpha=a)
    model_lasso.fit(X_train, y_train)
    coefs.append(model_lasso.coef_)
    MSE.append(mean_squared_error(y_val, model_lasso.predict(X_val)))

In [ ]:
RMSE = np.sqrt(MSE)
RMSE

In [ ]:
plt.plot(RMSE)

In [ ]:
model_lasso.set_params(alpha=0.000007)
model_lasso.fit(X_test, y_test)
TestRMSE_lasso = np.sqrt(mean_squared_error(y_test,model_lasso.predict(X_test)))
print(TestRMSE_lasso)

In [ ]:
tmp = model_lasso.predict(X_test)-y_test
print(np.median(tmp))
print(np.quantile(tmp,0.9))
print(np.quantile(tmp,0.99))
print(np.quantile(tmp,0.999))
print(np.amax(tmp))